In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import tensorflow as tf
from tensorflow import keras

In [3]:
df = pd.read_csv("/content/drive/MyDrive/asteroid/Asteroid_Updated.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,10,15,16,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
convertDict = {'diameter' : float}
df = df.astype(convertDict) 
df['diameter']

0         939.400
1         545.000
2         246.596
3         525.400
4         106.699
           ...   
839709        NaN
839710        NaN
839711        NaN
839712        NaN
839713        NaN
Name: diameter, Length: 839714, dtype: float64

In [6]:
df = df[df["diameter"].notnull()]

In [7]:
df['diameter'].isna().sum()

0

In [8]:
df =df.drop(['name'], axis=1)

In [9]:
df['data_arc'].fillna(df['data_arc'].mean(), inplace=True)

In [10]:
df['H'].fillna(df['H'].mean(), inplace=True)

In [11]:
df['albedo'].fillna(df['albedo'].mean(), inplace=True)

In [12]:
df['condition_code']=df['condition_code'].astype(float)
df['n_obs_used']=df['n_obs_used'].astype(float)
df['diameter']=df['diameter'].astype(float)

In [14]:
categorical_cols = ['neo','pha']
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
array_hot_encoded = ohe.fit_transform(df[categorical_cols]).toarray()
df_ohe = pd.DataFrame(array_hot_encoded, index=df.index, columns = ['N_neo','Y_neo', 'N_pha','Y_pha'])
df_other_cols = df.drop(columns=categorical_cols)
asteroid=pd.concat([df_ohe, df_other_cols], axis = 1)

In [15]:
asteroid = asteroid.drop(['class'], axis=1)

In [17]:
asteroid.dtypes

N_neo             float64
Y_neo             float64
N_pha             float64
Y_pha             float64
a                 float64
e                 float64
i                 float64
om                float64
w                 float64
q                 float64
ad                float64
per_y             float64
data_arc          float64
condition_code    float64
n_obs_used        float64
H                 float64
diameter          float64
extent             object
albedo            float64
rot_per           float64
GM                float64
BV                float64
UB                float64
IR                float64
spec_B             object
spec_T             object
G                 float64
moid              float64
n                 float64
per               float64
ma                float64
dtype: object

In [18]:
asteroid = asteroid.drop(['G', 'extent', 'rot_per','GM','BV','UB','IR','spec_B','spec_T'], axis=1)

In [19]:
asteroid.dtypes

N_neo             float64
Y_neo             float64
N_pha             float64
Y_pha             float64
a                 float64
e                 float64
i                 float64
om                float64
w                 float64
q                 float64
ad                float64
per_y             float64
data_arc          float64
condition_code    float64
n_obs_used        float64
H                 float64
diameter          float64
albedo            float64
moid              float64
n                 float64
per               float64
ma                float64
dtype: object

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
columns_to_be_used = ['data_arc', 'moid', 'q','a','albedo','n','H','diameter']
standard_df_temp = scaler.fit_transform(asteroid[columns_to_be_used])
list_columns = ['data_arc', 'moid', 'q','a','albedo','n','H','diameter']
new_asteroid_standard_scaled = pd.DataFrame(standard_df_temp, columns =columns_to_be_used)

In [21]:
new_asteroid_standard_scaled.isna().sum()

data_arc    0
moid        0
q           0
a           0
albedo      0
n           0
H           0
diameter    0
dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X = pd.DataFrame(new_asteroid_standard_scaled, columns = ['data_arc', 'moid', 'q','a','albedo','n','H'])
y = pd.DataFrame(new_asteroid_standard_scaled, columns = ['diameter'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from tensorflow.keras.layers import Dropout

In [25]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=12, activation='relu'))
ann.add(tf.keras.layers.Dense(units=12, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))
ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [26]:
ann.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3441/3441 [==============================] - 9s 2ms/step - loss: 0.3736
Epoch 2/100
3441/3441 [==============================] - 8s 2ms/step - loss: 0.1825
Epoch 3/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.1366
Epoch 4/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.1181
Epoch 5/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.0999
Epoch 6/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.1361
Epoch 7/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.0840
Epoch 8/100
3441/3441 [==============================] - 8s 2ms/step - loss: 0.0973
Epoch 9/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.0977
Epoch 10/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.0834
Epoch 11/100
3441/3441 [==============================] - 7s 2ms/step - loss: 0.0912
Epoch 12/100
3441/3441 [==============================] - 7s 2ms/step - lo

In [27]:
y_pred=ann.predict(X_test)

861/861 [==============================] - 1s 1ms/step


In [28]:
from sklearn.metrics import mean_squared_error, r2_score 
r2_score(y_test,y_pred)

0.8952052608240868